In [25]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from collections import defaultdict
import re
import pickle
import sys
from datetime import datetime

In [29]:
def get_data_id(comment_sect, data_id):
    '''
    data_id последнего комента верхнего уровня с текущей страницы для ссылки на следующую страницу
    '''      
    for bla in comment_sect:
        try:
            data_id = bla['data-ts']
        except KeyError:
            pass
        return data_id

def get_text(text, comment):
    '''
    Достать текст из тега
    '''
    for sect in text:
        comment += str(sect)
    comment = re.sub('<br/>', '', comment)
    return comment

def get_author(comment_sect, users):
    '''
    собрать комментировавших и отвечавших юзеров для каждого коммента и сколько раз каждый из них писал
    '''
    for author in comment_sect.findAll('a', {'class':'b-comments__item-author'}):
        if author:
            for each in author:
                for nick in each:
                    users[nick] += 1

def get_comment_text(comment_sect, each_comment):
    '''
    сбор непосредственно комментов и их дат
    '''
    for comment_itself in comment_sect.find('div', {'class':'b-comments__item m-first-level'}):
        if comment_itself:
            comment = ''
            date = comment_itself.find('div', {'class':'b-comments__item-date'})
            if date:
                date = date['datetime']
                text = comment_itself.find('div', {'class':'b-comments__item-text'})
                if text:
                    each_comment.append(get_text(text, comment))
    return date

def get_replies(comment_sect, each_comment, users):
    '''
    сбор ответов на комменты
    '''
    reply = ''
    reply_date = 0
    for reply_itself in comment_sect.find('div', {'class':'b-comments__list-answer'}):#'b-comments__list-items'
        if reply_itself:
            for for_reply_dates in reply_itself.findAll('div', {'class':'b-comments__item'}):
                if for_reply_dates:
                    reply_date = for_reply_dates['data-ts']
            text = reply_itself.findAll('div', {'class':'b-comments__item-text'})
            #date последнего ответа на коммент
            if text:                
                for sect in text:
                    each_comment.append(get_text(sect, reply))
                        
            next_reply_page = comment_sect.find('div', {'class':'b-comments__list-answer-load'})
            if next_reply_page:
                try:
                    next_reply_page['style']
                except KeyError:
                    next_replies(next_reply_page, users, reply_date, each_comment)

def next_replies(next_reply_page, users, reply_date, each_comment):
    '''
    перейти на страницу со всеми ответами, выкачать их и логины юзеров
    '''
    next_reply = ''
    parent_id = next_reply_page['data-parent_id']
    reply_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&parent_id='+parent_id+'&date='+str(reply_date)
    req_reply = requests.get(reply_link)
    soup_reply = BeautifulSoup(req_reply.text, 'lxml')
    
    for by_reply in soup_reply.findAll('div', {'class':'b-comments__item'}):
        if by_reply:
            reply_date = by_reply['data-ts']
            get_author(by_reply, users)
            text = by_reply.findAll('div', {'class':'b-comments__item-text'})
            if text:
                for sect in text:
                    each_comment.append(get_text(sect, next_reply))
            next_reply_page = by_reply.find('div', {'class':'b-comments__list-answer-load'})
            if next_reply_page:
                print('Noooooooooooooo')
                next_replies(next_reply_page, users, reply_date)
    return next_reply_page, users
        
def main_walker(comments_link, article_id, headline_comments_replies):
    '''
    Функция получает на вход ссылку, и собирает в массив кортежи из комметария, 
    ответов на него и словаря с пользователями и количеством комментов/ответов от них
    '''
    req = requests.get(comments_link)
    soup = BeautifulSoup(req.text, 'lxml')
    data_id = 0
    for comment_sect in soup.findAll('div', {'class':'b-comments__list-tree'}):
        data_id = get_data_id(comment_sect, data_id) #data_id последнего комента верхнего уровня с текущей страницы для ссылки на следующую страницу
        date = ''
        each_comment = []
        date = get_comment_text(comment_sect, each_comment) #сбор непосредственно комментов и их дат
        users = defaultdict(int)
        get_author(comment_sect, users)
        get_replies(comment_sect, each_comment, users)
        headline_comments_replies.append((date, list(set(each_comment)), users))
        
    next_p = soup.find('div', {'class':'b-comments__list-answer'})
    if next_p:
        comments_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&sort=desc&date='+data_id
        main_walker(comments_link, article_id, headline_comments_replies)

start_time = datetime.now()
trial = ['https://ria.ru/world/20171010/1506569366.html']
polemic_comments = []
for cortege in to_get_comments:
    polemic_cortege = []
    print('Now processing cluster №\t', cortege[0])
    for headline in cortege[1]:
        link = cortege[1][headline]
        article_id = link[-15:-5] 
        comments_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&sort=desc&add_class=m-main-list'
        headline_comments_replies = []
        main_walker(comments_link, article_id, headline_comments_replies)
        polemic_cortege.append((headline, cortege[1][headline], headline_comments_replies))
    polemic_comments.append((cortege[0], polemic_cortege))

sys.setrecursionlimit(100000)
with open('polemic_comments.pkl', 'wb') as fp:
    pickle.dump(polemic_comments, fp)
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))    

Now processing  6  cluster
Now processing  7  cluster
Now processing  12  cluster
Now processing  16  cluster
Now processing  20  cluster
Now processing  21  cluster
Now processing  24  cluster
Now processing  30  cluster
Now processing  33  cluster
Now processing  34  cluster
Duration: 0:06:25.877341


In [31]:
with open('polemic_comments.txt', 'w', encoding='utf-8') as f:
    f.write(str(polemic_comments))
#headline_comments_replies

In [105]:
with open ('polemic_comments.pkl', 'rb') as fp:
    polemic_comments = pickle.load(fp)

In [126]:
"достать все комментарии в каждом кластере"
only_comments_by_cluster = {}
for clstr in polemic_comments:
    clstr_comments = []
    cntr_articles = 0
    for article in clstr[1]:
        cntr_articles += 1
        comments_arr = article[2]
        #print(len(comments_arr))
        if len(comments_arr) != 0:
            #print(comments_arr)
            for inner_tuple in comments_arr:
                #print(inner_tuple)
                for comment_text in inner_tuple[1]:
                    clstr_comments.append(comment_text)
    cntr_comments = len(clstr_comments)
    print(clstr[0], cntr_articles, cntr_comments, cntr_comments/cntr_articles)
    only_comments_by_cluster[clstr[0]] = clstr_comments

6 18 74 4.111111111111111
7 28 87 3.107142857142857
12 13 30 2.3076923076923075
16 7 31 4.428571428571429
20 32 443 13.84375
21 20 248 12.4
24 8 47 5.875
30 68 1521 22.36764705882353
33 17 985 57.94117647058823
34 16 334 20.875


In [104]:
"достать всех пользователей"
all_users = set()
for clstr in polemic_comments:
    #clstr_comments = []
    #cntr_articles = 0
    for article in clstr[1]:
        #cntr_articles += 1
        comments_arr = article[2]
        #print(len(comments_arr))
        if len(comments_arr) != 0:
            #print(comments_arr)
            for inner_tuple in comments_arr:
                #print(inner_tuple)
                for user in inner_tuple[2].keys():
                    all_users.add(user)
    #cntr_comments = len(clstr_comments)
    #print(clstr[0], cntr_articles, cntr_comments, cntr_comments/cntr_articles)
    #only_comments_by_cluster[clstr[0]] = clstr_comments
len(all_users)

1948

In [77]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
import re
import pymorphy2
from datetime import datetime

fname = 'noStopLemma_PV-DBOW_wrd-vec_1it_2win_6mincount_alpha25-25_sz80.model'

model = Doc2Vec.load(fname)

morph = pymorphy2.MorphAnalyzer()

C:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [78]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    txt = f.read().split('\n')
stw = set(txt)

In [87]:
lines

['бог даст - и во втором победит.',
 'в не всякого сомнения, победа на выборах за марин ле пен, какие бы козни её не чинили...',
 'как с трампом. 61% из специальной выборки, исключающей 100% тех, кто точно против.',
 'макрон - президент франции! трамп - ненаш! черногория в нато! обидно, да, рос-патриоты?))))',
 'ещё не вечер . а этим заказным опросам грош цена в базарный день .',
 'ага, верю-верю! клинтоншу тоже, согласно опросов, поддерживало большинство, и победа была гарантирована. и где теперь те опросники, и их опросы- прогнозы?))',
 'после выборов,если будет ставленник пиндосии макарон, то французов мне жалко. и больше они для меня не будут представлять великую францию.',
 'сергей викторов, ну, зачем ты так жестоко с ними? они же очень сильно расстроятся, что не представляют для рос-патриота сереги великую францию)))))',
 'до уличного светила, что называется, это дело народа франции.',
 'опрос про клинтон вспомнился...',
 'александр михайлов,  про париж точно знаешь? часто там бы

In [131]:
clustersizes = []

def distanceToCentroid():
    for i in range(0, NUM_CLUSTERS):
        clustersize = 0
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                clustersize+=1
        clustersizes.append(clustersize)
        dist = 0.0
        centr = kclusterer.means()[i]
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                dist += pow(nltk.cluster.util.cosine_distance(vectors[j], centr),2)/clustersize
        dist = math.sqrt(dist)
        print("distance cluster: "+str(i)+" RMSE: "+str(dist)+" clustersize: "+str(clustersize))

def nClosestToCentroid(cluster_id, n):
    #clustersize = len(get_titles_by_cluster(cluster_id))
    centr = kclusterer.means()[cluster_id]
    distances = []
    for j in range(0, len(assigned_clusters)):
        if (assigned_clusters[j] == cluster_id):
            distances.append((used_lines[j], nltk.cluster.util.cosine_distance(vectors[j], centr)))
    distances = sorted(distances, key=lambda tup: tup[1])
    return distances[:n]


def get_titles_by_cluster(id):
    list = []
    for x in range(0, len(assigned_clusters)):
        if (assigned_clusters[x] == id):
            list.append(used_lines[x])
    return list

def get_topics(titles):
    from collections import Counter
    words = [preprocess_document(x) for x in titles]
    words = [word for sublist in words for word in sublist]
    #filtered_words = [word for word in words if word not in stw]
    count = Counter(words)
    print(count.most_common()[:5])


def cluster_to_topics(id):
    get_topics(get_titles_by_cluster(id))
    

NUM_CLUSTERS = 5

def preprocess(str):
    # remove links
    str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    return str


def preprocess_document(text):
    #text = preprocess(text)
    fixedNoStop = []
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    return fixedNoStop

start_time = datetime.now()

#data = <sparse matrix that you would normally give to scikit>.toarray()

#corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
#for cluster in only_comments_by_cluster:
#    print(len(only_comments_by_cluster[cluster]))
#lines = ['Бог даст - и во втором победит.', 'В не всякого сомнения, победа на выборах за Марин Ле Пен, какие бы козни её не чинили...', 'Как с Трампом. 61% из специальной выборки, исключающей 100% тех, кто точно против.', 'Макрон - Президент Франции! Трамп - ненаш! Черногория в НАТО! Обидно, да, рос-патриоты?))))', 'ещё не вечер . А этим заказным опросам грош цена в базарный день .', 'Ага, верю-верю! Клинтоншу тоже, согласно опросов, поддерживало большинство, и победа была гарантирована. И где теперь те опросники, и их опросы- прогнозы?))', 'После выборов,если будет ставленник пиндосии Макарон, то французов мне жалко. И больше они для меня не будут представлять Великую Францию.', 'Сергей Викторов, Ну, зачем ты так жестоко с ними? Они же очень сильно расстроятся, что не представляют для рос-патриота сереги Великую Францию)))))', 'До уличного светила, что называется, это дело народа Франции.', 'Опрос про Клинтон вспомнился...', 'Александр Михайлов,  Про Париж точно знаешь? Часто там бываешь, рос-патриот?))))', 'Александр Михайлов, Сколько боярышника принял, рос-патриот,  чтобы в соседнем дворе "Париж" увидеть?))))']
lines = only_comments_by_cluster['6']
for line in range(len(lines)):
    lines[line] = lines[line].lower()
#lines = corpus.read().lower().split('\r\n')
#count = len(lines)

vectors = []

print("inferring vectors")
duplicate_dict = {}
used_lines = []
for i, t in enumerate(lines):
    if t not in duplicate_dict:#i % 2 == 0 and
        duplicate_dict[t] = True
        used_lines.append(t)
        vectors.append(model.infer_vector(preprocess_document(t)))

print("done")



kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=10)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
print('Cluster assigning done!')
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

inferring vectors
done
Cluster assigning done!
Duration: 0:00:00.687492


In [132]:
for clstr in range(NUM_CLUSTERS):
    #cluster_to_topics(clstr)
    #print(get_titles_by_cluster(clstr))
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('наверное, для демократии такие дебаты обязательны.и понимаешь (в какой-то мере) с кем имеешь дело, и политики узнают реальные отличия позиций друг друга.только наши, к сожалению, с прямым диалогом и открытыми вопросами не справятся. да и не нужно им это: воровать мешает.поэтому и игнорируют такие дебаты.', 0.22148576339227521), ('ага, верю-верю! клинтоншу тоже, согласно опросов, поддерживало большинство, и победа была гарантирована. и где теперь те опросники, и их опросы- прогнозы?))', 0.24988775907149763), ('dlinny_nos, тут не говорить надо, а действовать.', 0.27439817795254129), ('ну это ещё бабушка надвое сказала.трамп пример.клинтонше тоже сулили победу и все её сторонники  решили что и без его голоса она наберёт достаточно. а получилось наоборот.так что это нагнетание с макароном может сыграть шуточку.', 0.27934439057138183), ('лишь бы не получилось как с трампом, запевшим теперь другое по отношению к нам.', 0.30556859483454846)]
####

[('в не всякого сомнения, победа на выбора